# Regularization with Weight Decay

In [56]:
import numpy as np
import pandas as pd
from __future__ import division

In [69]:
# Load training and testing data
train = pd.read_csv('in_dta.txt', delim_whitespace=True, header=None, names=['x1', 'x2', 'y'])
test = pd.read_csv('out_dta.txt', delim_whitespace=True, header=None, names=['x1', 'x2', 'y'])

def transform(data):
    """Transform the data so we can apply regression to a non linear function.
    phi = 1, x1, x2, x1^2, x2^2, x1x2, |x1 − x2|, |x1 + x2|)
    """
    data.loc[:, 'x0'] = pd.Series(1, index=data.index)
    data.loc[:, 'x3'] = data.x1.map(lambda x: x**2)
    data.loc[:, 'x4'] = data.x2.map(lambda x: x**2)
    data.loc[:, 'x5'] = data.x1 * data.x2
    data.loc[:, 'x6'] = (data.x1 - data.x2).map(lambda x: np.abs(x))
    data.loc[:, 'x7'] = (data.x1 + data.x2).map(lambda x: np.abs(x))
    # Sort columns into the correct order
    data = data.reindex_axis(sorted(data.columns), axis=1)
    return data
    
def lin_regression(data):
    """Apply linear regression the dataset
    w = (x^Tx)^-1 * x^T * y"""
    x = np.matrix(data.drop(labels='y', axis=1).as_matrix())
    y = np.matrix(data.y.as_matrix())
    w = (x.T * x).I * x.T * y.T
    return w

def lin_regression_weights(data, k):
    """Apply linear regression with weight decay"""
    x = np.matrix(data.drop(labels='y', axis=1).as_matrix())
    y = np.matrix(data.y.as_matrix())
    product = x.T * x
    lambda_I = np.matrix((10**k) * np.identity(len(product)))
    w = (product + lambda_I).I * x.T * y.T
    return w

def error(data, w):
    """Given transformed out of sample data and the weight vector, calculate the error between g and f 
    (use the proportion of misclassified points)"""
    x = np.matrix(data.drop(labels='y', axis=1).as_matrix())
    y = np.matrix(data.y.as_matrix()).T
    y_predicted = x * w
    count = 0
    for i in range(len(y)):
        if np.sign(y_predicted[i, 0]) != y[i, 0]:
            count += 1
    return count / len(y_predicted)

# No weight decay
train_t = transform(train)
test_t = transform(test)
w = lin_regression(train_t)
print "In sample error: {}".format(error(train_t, w))
print "Out of sample error: {}".format(error(test_t, w))

In sample error: 0.0285714285714
Out of sample error: 0.084


In [75]:
# With weight decay lambda = 10 ^ k
w = lin_regression_weights(train_t, k=-2)
print "In sample error: {}".format(error(train_t, w))
print "Out of sample error: {}".format(error(test_t, w))

In sample error: 0.0285714285714
Out of sample error: 0.084


In [78]:
min = (100, 3)
for i in range(-20, 20, 1):
    w = lin_regression_weights(train_t, i)
    e = error(test_t, w)
    if e < min[0]:
        min = (e, i)
        
print min

(0.056, -1)
